# TP : LLM, Langchain et RAG
## Installation des dépendances

### Prérequis : 
- Mettre la clé d'API OpenAI en variable d'environnement
- Installer les packages suivants : 

In [ ]:
!pip install langchain openai datasets transformers PyMuPDF langchain_community langchain_openai langchain_text_splitters faiss-cpu sentence-transformers

## LLM
### OpenAI
**Objectif** : faire une requête au modèle 'gpt4o-mini' via la librairie d'OpenAI et récupérer : 
- la réponse 
- le nombre de tokens d'entrée
- le nombre de tokens de sorties

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import os

In [ ]:
def ask_gpt(question: str):
    """
    Prend une question et retourne la réponse ainsi que les KPIs d'usage
    
    Args: 
        question(str) : question à poser
    """
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"{question}"}
      ]
    )
    
    return(completion.choices[0].message.content, completion.usage)

answer, usage = ask_gpt("Qu'est ce qu'un LLM ?")

In [ ]:
usage

In [ ]:
answer

## RAGS from Scracth
**Objectif** : Développer, from scratch, une architecture RAG en 4 modules: 
- Lecture et découpage d'un document PDF
- Générer les embeddings
- Récupérer les passages pertinents en fonction d'une requête
- Générer une réponse à partir de la question et des passages pertinents

In [ ]:
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import PyPDF2
import os

In [ ]:
# Module 1 : Lecture et découpage d’un document PDF
def read_and_split_pdf(pdf_path, chunk_size=500):
    """
    Lit un fichier PDF et découpe le contenu en morceaux de texte.
    
    Args:
    pdf_path (str): Chemin du fichier PDF.
    chunk_size (int): Taille des morceaux (en nombre de caractères).
    
    Returns:
    list: Liste des morceaux de texte.
    """
    text_chunks = []
    with open(pdf_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        full_text = ""
        for page in reader.pages:
            full_text += page.extract_text()
        text_chunks = [full_text[i:i+chunk_size] for i in range(0, len(full_text), chunk_size)]
    return text_chunks

# Tester avec un fichier PDF
pdf_chunks = read_and_split_pdf("../Data/Cours_LLM.pdf")
print(f"Nombre de morceaux générés : {len(pdf_chunks)}")
print("Extrait du premier morceau :\n", pdf_chunks[0])

In [ ]:
# Module 2 : Génération des embeddings et stockage avec FAISS

def generate_embeddings(chunks):
    """
    Génère les embeddings pour chaque morceau de texte.
    
    Args:
        chunks (list): Liste des morceaux de texte.
    
    Returns:
        np.array: Tableau numpy des embeddings.
    """
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(chunks, show_progress_bar=True)
    return np.array(embeddings)

def create_faiss_index(embeddings):
    """
    Crée un index FAISS à partir des embeddings.
    
    Args:
    embeddings (np.array): Tableau numpy des embeddings.
    
    Returns:
    faiss.IndexFlatL2: Index FAISS.
    """
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

# Générer les embeddings et créer l'index
embeddings = generate_embeddings(pdf_chunks)
faiss_index = create_faiss_index(embeddings)

print(f"Nombre de vecteurs dans l'index : {faiss_index.ntotal}")

In [ ]:
# Module 3 : Recherche des passages pertinents avec une requête
def query_faiss_index(query, index, model, chunks, k=5):
    """
    Recherche les passages les plus pertinents dans l'index FAISS.
    
    Args:
    query (str): La requête de l'utilisateur.
    index (faiss.IndexFlatL2): L'index FAISS.
    model (SentenceTransformer): Modèle pour générer les embeddings.
    chunks (list): Liste des morceaux de texte.
    k (int): Nombre de passages à retourner.
    
    Returns:
    list: Liste des passages pertinents.
    """
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k)
    return [chunks[i] for i in indices[0]]

# Tester avec une requête
query = "Qu'est ce qu'un RAG ?"
relevant_chunks = query_faiss_index(query, faiss_index, SentenceTransformer('all-MiniLM-L6-v2'), pdf_chunks)
print("Passages pertinents :")
for chunk in relevant_chunks:
    print("-" * 80)
    print(chunk)

In [ ]:
# Module 4 : Générer une réponse avec un LLM
def generate_answer(query, relevant_chunks):
    """
    Génère une réponse à partir de la question et des passages pertinents.
    
    Args:
    query (str): La question posée.
    relevant_chunks (list): Passages pertinents.
    
    Returns:
    str: Réponse générée.
    """
    context = "\n".join(relevant_chunks)
    prompt = f"Voici des informations utiles :\n{context}\n\nQuestion : {query}\nRéponse :"
    
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": "Tu es un assistant RAG spécialisé dans la rédaction de réponse."},
        {"role": "user", "content": f"{prompt}"}
      ]
    )
    
    return(completion.choices[0].message.content)

# Tester la génération de réponse
response = generate_answer(query, relevant_chunks)
print("Réponse générée :")
print(response)


## Langchain
**Objectif** : faire une requête au modèle 'gpt4o-mini' via la librairie Langchain et récupérer la réponse 

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
def ask_gpt_langchain(question):
    """
    Prend une question et retourne la réponse générée via langchain
    
    Args: 
        question(str) : question à poser
    """
        
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    prompt_template = PromptTemplate.from_template(question)
    chain = LLMChain(llm=llm, prompt=prompt_template)
    
    return(chain.invoke({}))

anwser = ask_gpt_langchain("Qu'est ce qu'un transformer ?")
print(anwser)

## Lecture d'un PDF et utilisation dans RAG avec LangChain
**Objectif** : Construire une architecture RAG avec langchain via 3 modules : 
1 - récupération du document via PyPDFLoader
2 - génération du retriever après avoir split le document
3 - Création de la chaîne de réponse à la question


In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_openai import OpenAIEmbeddings

In [ ]:
def get_docs(pdf_path):
    """
    Charge et extrait le contenu textuel d'un fichier PDF.

    Args:
        pdf_path (str): Chemin vers le fichier PDF à charger.

    Returns:
        list: Liste contenant le contenu textuel de chaque page du PDF.
    """
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    return(docs)

In [ ]:
def get_retriever(docs):
    """
    Crée un récupérateur (retriever) basé sur les embeddings pour une recherche de similarité.

    Args:
        docs (list): Liste des documents à traiter

    Returns:
        BaseRetriever
    """
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, 
                                                   chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = InMemoryVectorStore.from_documents(documents=splits, 
                                                     embedding=OpenAIEmbeddings())
    return vectorstore.as_retriever()

In [ ]:
def generate_answer(question, retriever) :
    """
    Génère une réponse à une question en utilisant une architecture RAG.

    Args:
        question (str): La question posée.
        retriever (BaseRetriever): Un récupérateur permettant de chercher le contexte pertinent
                                   pour répondre à la question.

    Returns:
        str: La réponse générée par le modèle, basée sur le contexte récupéré. 
    """
    system_prompt = ("You are an assistant for question-answering tasks. "
                     "Use the following pieces of retrieved context to answer "
                     "the question. If you don't know the answer, say that you "
                     "don't know. Use three sentences maximum and keep the "
                     "answer concise."
                     "\n\n"
                     "{context}")
    
    prompt = ChatPromptTemplate.from_messages([("system", system_prompt),
                                               ("human", "{input}"),])
    
    llm = ChatOpenAI(model="gpt-4o-mini")
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)
    
    results = rag_chain.invoke({"input": f"{question}"})
    
    return(results)

In [ ]:
docs = get_docs('../Data/Cours_LLM.pdf')
retriever = get_retriever(docs)
results = generate_answer("Qu'est ce qu'un RAG ?", retriever)

In [ ]:
print(results["answer"])

In [ ]:
print(results["context"])

# Ollama

In [ ]:
import ollama
response = ollama.chat(model='mistral', messages=[
   {
     'role': 'user',
     'content': 'Talk about Data Science',
   },
 ], stream=True)

for chunk in response:    
    print(chunk["message"]["content"], end='', flush=True)